<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/tomato_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

토마토 가격 예측 

In [1]:
!pip uninstall --y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=fa4ec33e0a43d4d3c018e4e91ce90372aad6b5f5722e18f83447218efcc39a4d
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 29 12:20 kaggle.json


In [4]:
!kaggle competitions download -c ai-tomato

  0% 0.00/26.9k [00:00<?, ?B/s]
100% 26.9k/26.9k [00:00<00:00, 21.7MB/s]


In [5]:
!unzip ai-tomato.zip

Archive:  ai-tomato.zip
  inflating: submit_sample.csv       
  inflating: test_set.csv            
  inflating: training_set.csv        


In [6]:
import pandas as pd 
import numpy as np
import torch 
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [8]:
train = pd.read_csv('training_set.csv')
print(train.head(10))
print(train.info())


       date  avg temp  min temp  max temp  rain fall  wind  wind chill  price
0  20100104      -5.9      -8.0      -3.4       14.2   3.0        -9.7   2500
1  20100105      -9.9     -12.3      -7.0        0.0   2.8       -14.0   2500
2  20100106     -11.2     -13.3      -8.1        0.0   2.0       -14.2   2300
3  20100107     -10.1     -13.6      -5.5        0.0   1.7       -12.6   2200
4  20100108      -8.0     -11.8      -3.3        0.0   1.8       -10.4   2000
5  20100111      -5.2      -8.3      -1.3        0.0   2.2        -7.5   2000
6  20100112     -10.3     -12.9      -7.3        0.0   2.9       -14.6   2300
7  20100113     -13.2     -15.3     -10.7        0.0   3.1       -18.8   2200
8  20100114      -9.1     -14.4      -4.1        0.0   1.8       -12.7   2600
9  20100115      -5.0      -7.5      -0.6        0.3   2.4        -7.9   2400
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2120 entries, 0 to 2119
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dty

In [43]:
learning_rate = 0.001
training_epoch = 2000
batch_size = 40 

In [44]:
x_train = train.iloc[:,1:-1]
y_train = train.iloc[:,[-1]]

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

print(x_train.shape)
print(y_train.shape)

torch.Size([2120, 6])
torch.Size([2120, 1])


In [45]:
train_dataset = torch.utils.data.TensorDataset(x_train,y_train)

data_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last=True)



layer 5개 , kaiming uniform 초기화

In [46]:
linear1 = nn.Linear(6,4,bias=True)
linear2 = nn.Linear(4,4,bias=True)
linear3 = nn.Linear(4,4,bias=True)
linear4 = nn.Linear(4,4,bias=True)
linear5 = nn.Linear(4,1,bias=True)

nn.init.kaiming_uniform_(linear1.weight)
nn.init.kaiming_uniform_(linear2.weight)
nn.init.kaiming_uniform_(linear3.weight)
nn.init.kaiming_uniform_(linear4.weight)
nn.init.kaiming_uniform_(linear5.weight)

relu = nn.SELU()

In [47]:
model = nn.Sequential(
    linear1,relu,
    linear2,relu,
    linear3,relu,
    linear4,relu,
    linear5
).to(device)

In [48]:
loss = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [49]:
total_batch = len(data_loader)

for epoch in range(training_epoch):
    avg_cost = 0
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis,Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('epoch {:.4f} , cost = {:.6f}'.format(epoch,avg_cost))
print('learning finished!')

epoch 0.0000 , cost = 8485569.000000
epoch 1.0000 , cost = 8457462.000000
epoch 2.0000 , cost = 8381214.000000
epoch 3.0000 , cost = 8212285.000000
epoch 4.0000 , cost = 7868885.000000
epoch 5.0000 , cost = 7263739.500000
epoch 6.0000 , cost = 6368924.500000
epoch 7.0000 , cost = 5304735.000000
epoch 8.0000 , cost = 4472637.000000
epoch 9.0000 , cost = 4104556.250000
epoch 10.0000 , cost = 3950030.000000
epoch 11.0000 , cost = 3821644.750000
epoch 12.0000 , cost = 3686691.000000
epoch 13.0000 , cost = 3542024.750000
epoch 14.0000 , cost = 3390354.000000
epoch 15.0000 , cost = 3231972.000000
epoch 16.0000 , cost = 3070249.750000
epoch 17.0000 , cost = 2908677.750000
epoch 18.0000 , cost = 2751725.500000
epoch 19.0000 , cost = 2603533.750000
epoch 20.0000 , cost = 2459710.750000
epoch 21.0000 , cost = 2320600.500000
epoch 22.0000 , cost = 2186303.250000
epoch 23.0000 , cost = 2055917.500000
epoch 24.0000 , cost = 1922423.500000
epoch 25.0000 , cost = 1803086.625000
epoch 26.0000 , cost =

In [50]:
test = pd.read_csv('test_set.csv')
print(test.info())
test = test.iloc[:,1:-1]
test = np.array(test)
test = torch.FloatTensor(test).to(device)

with torch.no_grad():
    predict = model(test)
predict

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        101 non-null    int64  
 1   avg temp    101 non-null    float64
 2   min temp    101 non-null    float64
 3   max temp    101 non-null    float64
 4   rain fall   101 non-null    float64
 5   wind        101 non-null    float64
 6   wind chill  101 non-null    float64
 7   Unnamed: 7  0 non-null      float64
dtypes: float64(7), int64(1)
memory usage: 6.4 KB
None


tensor([[1615.6046],
        [1947.7316],
        [1864.0270],
        [2048.9602],
        [1961.5825],
        [1985.7471],
        [2174.5815],
        [1847.1421],
        [2111.6567],
        [1881.0664],
        [1869.3419],
        [1953.7471],
        [2044.2358],
        [2042.3242],
        [1724.1357],
        [2176.5369],
        [2162.8611],
        [2172.0723],
        [2194.8599],
        [2159.4143],
        [2161.6873],
        [2174.9456],
        [2170.3047],
        [2340.6848],
        [2308.6428],
        [2188.5596],
        [2393.9502],
        [2409.2507],
        [2618.0688],
        [2491.4956],
        [2178.9521],
        [2162.3547],
        [2420.4402],
        [2495.4871],
        [2177.5139],
        [2136.5815],
        [2185.3574],
        [2764.9653],
        [2913.8015],
        [2952.2312],
        [2943.0725],
        [2721.5469],
        [2124.1687],
        [3063.0061],
        [3107.9841],
        [3270.8010],
        [3159.7888],
        [3028

In [51]:
correct_prediction = predict.cpu().numpy().reshape(-1,1)
result = pd.read_csv('submit_sample.csv')

In [52]:
for i in range(len(correct_prediction)):
    result['expected'][i] = correct_prediction[i]
    

In [53]:
result.to_csv('submit.csv',index=False)

In [54]:
!kaggle competitions submit -c ai-tomato -f submit.csv -m "14010974_이기택_기말고사파트2-1"

100% 811/811 [00:07<00:00, 106B/s]
Successfully submitted to SejongAI.텀프로젝트.[토마토 가격 예측]